# 智能体初步

## 这节课会带给你

- 🌹 了解基本的 Langchain 开发
- 🌹 初步接触智能体
- 🌹 深入了解工具回调智能体的内部逻辑

## 一、大模型开发

### 1、访问智谱大模型官网

- 智谱API：[访问智谱官网](https://maas.aminer.cn)
- 申请Key：[获得 API keys](https://maas.aminer.cn/usercenter/apikeys

### 2、在 langchain 中，大模型的一般用法回顾

- Langchain的LCEL初步用法
- 构造消息模板：对照官网
- 解析：提取文本、JSON或代码块
- invoke和stream方法
- 转化为API

#### ✍️ 使用 ChatZhipuAI

In [ ]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
# LLM
from langchain_zhipu import ChatZhipuAI

llm = ChatZhipuAI()

In [ ]:
# invoke
text = "你知道我家的猫是什么颜色的吗？"
llm.invoke(text)

In [ ]:
# stream
for chunk in llm.stream(text):
    print(chunk)

<div class="alert alert-warning">
    <b>⚠️ 思考</b><br>
    langchain 支持8个标准化方法，都在什么场景下使用？
</div>

**支撑 LCEL 承诺主要是依靠这8个方法：**

- invoke：最简单
- batch: 后台批量
- stream：让用户体验流式输出
- ainvoke / astream / abatch: 实现异步体验
- astream_log / astream_events: 从Runable、LCEL、智能体、langgraph等提取特定的流式输出

#### ✍️ LCEL：Prompt + LLM + OutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate.from_template("你知道{what}是什么颜色吗?")
chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"what": "太阳"}):
    print(chunk, end="|", flush=True)

<div class="alert alert-success">
    <b>⚠️ langchain 的核心框架</b><br>
    组件方法标准化（Runnable） + 调度框架（LCEL / 智能体 / Langgraph）
</div>

## 二、智能体开发概述

### 1、关于智能体的几个问题

- 什么是工具回调？
- 什么是智能体？
- 都有哪些常见的智能体类型？

### 2、大模型中的工具回调

#### ❤️ 官网接口

- [智谱官网的工具回调说明](https://maas.aminer.cn/dev/howuse/functioncall)
- [智谱官网的接口说明](https://maas.aminer.cn/dev/api#glm-4)
- [OpenAI的工具回调说明](https://platform.openai.com/docs/guides/function-calling)

#### ✍️ 使用 langchain 定义工具

<div class="alert alert-warning">
<b>注意：</b><br>
    
1. OpenAI/ZhipuAI使用的Tools名称类似于一个函数命名，必须使用下划线或ASCII码，**不能使用中文**！<br>
但描述部份可以使用中文。
2. 定义工具时，请**至少包含一个参数**（即使你不使用这个参数），否则调用 GPT 时可能会抛出异常
</div>

In [ ]:
import json
import random

from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

@tool
def where_to_lookup(target: str = "cat") -> str:
    """
    因为家里比较大，需要首先使用这个工具猜测可以躲藏的哪个地方。

    args:
    - target 可以是cat或其他
    """

    if target == "cat":
        places = ["床底下", "书架中", "阳台附近"]
        return places[2]
        # return random.choice(places)
    else:
        return "我不知道"

@tool
def lookup_result(place: str) -> str:
    """如果你猜测到了一个地方，可以使用这个工具去仔细查看否躲藏在这里"""

    if "床底" in place:  # For under the bed
        return "发现几只臭袜子和鞋，但没发现目标"
    elif "书架" in place:  # For 'shelf'
        return "发现一些漫画书、图册和铅笔，但没发现目标"
    elif "阳台" in place:
        return "我找到了"
    else:  # 如果智能体决定找其他地方
        return "找到屋外了，但没有任何发现"

#### ✍️ 便捷转换：convert_to_openai_tool

In [ ]:
print(json.dumps(convert_to_openai_tool(where_to_lookup), indent=2, ensure_ascii=False))

#### ✍️ 调用包含工具的 ZhipuAI 实例

In [ ]:
resp = llm.invoke(
    "猫藏在哪里？",
    tools=[
        convert_to_openai_tool(where_to_lookup),
        convert_to_openai_tool(lookup_result)
    ])
print(resp)

In [ ]:
where_is_cat_hiding.invoke({"idea":"猫可能会藏在哪里呢？"})

### 3、实现OpenAI风格的工具回调智能体

In [ ]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

def create_openai_executor(llm, tools):
    """
    使用openai智能体定义一个应用
    """
    # 定义 prompt
    prompt = hub.pull("hwchase17/openai-tools-agent")
    # 定义 Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=False)

    return executor

lookup_cat = create_openai_executor(ChatZhipuAI(), [where_to_lookup, lookup_result])

In [ ]:
lookup_cat.invoke({"input": "我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。（注意，你每一步思考都必须使用工具）你要努力去找哦！"})

In [ ]:
from langchain_openai import ChatOpenAI
lookup_cat = create_openai_executor(ChatOpenAI(), [where_to_lookup, lookup_result])
lookup_cat.invoke({"input": "我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！"})




....



**这里暂停探索**：我们先不要手工实现，而是转而尝试 langchain 封装好的工具回调智能体。

### 4、查看 OpenAI 智能体的提示语

#### ❤️ 从 Langsmith 的 hub 下载 hwchase17/openai-tools-agent

[🔗 查看 hub.pull("hwchase17/openai-tools-agent")](https://smith.langchain.com/hub/hwchase17/openai-tools-agent)

#### ❤️ 等价的自定义 Prompt 模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的助手"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [ ]:
prompt.invoke({
    "input": "我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！",
    "chat_history": [],
    "agent_scratchpad": []
})

### 5、OpenAI 智能体的运行过程

#### ✍️ 简单执行：invoke

In [ ]:
# invoke
lookup_cat.invoke({"input":"我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！"})

#### ✍️ 流式执行：stream 和 事件流

**使用stream：**

In [ ]:
# stream
for s in lookup_cat.stream({"input":"我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！"}):
    print(s)

#####  还是要使用事件流：astream_events

**看看方法 astream_events 的能力：**

In [ ]:
# astream_events
async for e in lookup_cat.astream_events({"input":"我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！"}, version="v1"):
    print(e['name'], e['tags'], e['event'])

#### ✍️ 事件流执行：观察与大模型的关键事件

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [ ]:
async for e in lookup_cat.astream_events({"input":"我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])
        # print("\n", e)

#### ❤️ 总结智能体的运行过程：请对照官网中的接口定义体会

- STEP-1 请求智能体（langchain -> LLM）: 发送带有Tools的请求
- STEP-2 智能体解析（LLM -> langchain）: 收到Tools-Calling消息
- STEP-3 调用本地工具（langchain）: 调用工具
- STEP-4 重新请求智能体（langchain） -> LLM）: 提交调用结果
- STEP-5 智能体最终生成（LLM -> langchain）: 大模型重新生成最终结果，或重复 STEP-3

### 6、通过单步实验来研究智能体的调用结果

#### 🌹 选修：你也可以尝试阅读 AgentExcutor 源码

[🔗 查看 AgentExcutor 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/agent.py#L1413-L1458)

<div class="alert alert-warning">
    <b>⚠️ 注意：本节内容非常重要 ！！！</b><br>
    本节内容是最核心部份，是对智能体的运行逻辑的一步一步剖析，是真正掌握复杂智能体的基础。
</div>

#### （1）STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai风格智能体的提示语
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的助手"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

## 定义openai风格智能体
llm = ChatZhipuAI(tools=[convert_to_openai_tool(t) for t in [where_to_lookup, lookup_result]])
agent = prompt | llm

## 智能体的短期记忆
scratchpad = []

#### （2）STEP-2 智能体解析（LLM -> langchain）: 收到Tools-Calling消息

In [ ]:
# 给智能体的输入
input = "我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。（注意每一步你都应该使用工具）你要努力去找哦！"

# 智能体中大模型的第一次反馈
resp_llm = agent.invoke({"input": input, "chat_history": [], "agent_scratchpad": scratchpad})
resp_llm

#### （3）STEP-3 调用本地工具（langchain）: 调用工具

In [ ]:
# 获得回调工具名称
resp_llm.additional_kwargs["tool_calls"][0]["function"]["name"]

In [ ]:
# 获得回调工具参数
resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"]

In [ ]:
# 注意输出结果中的JSON是被引号包围的字符串，需要使用json工具提取
import json
tool_args = json.loads(resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"])
tool_args

In [ ]:
# 调用工具
resp_tool = where_to_lookup.invoke(tool_args)
resp_tool

[🔗 查看 langhain官方解析openai回调参数的方法](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/openai_tools.py)

#### （4）STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果

In [ ]:
# 将输入、大模型返回和工具结果填充到短期记忆链中
scratchpad.append(input)
scratchpad.append(resp_llm)
scratchpad.append(resp_tool)
scratchpad

In [ ]:
# 通过索引看得更真切一些
scratchpad[2]

#### （5）STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

In [ ]:
# 结合短期记忆，再次调用大模型
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": scratchpad})
resp_llm

#### （6）STEP-6 （重复上面的步骤）继续调用工具，直到结束

## 三、总结

- 🌹 Langchain 开发的基本范式： Prompt+LLM+OutputParser
- 🌹 初步接触智能体：工具回调智能体
- 🌹 智能体的内部运行逻辑：LLM推理 -> 调用工具 -> LLM推理 -> ... -> LLM认为结束

**再次复习调用过程：**

In [ ]:
async for e in lookup_cat.astream_events({"input":"我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])
        # print("\n", e)